<a href="https://colab.research.google.com/github/olga-terekhova/indoor-co2-forecast/blob/main/notebooks/ExtractHomeAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract source data from Home Assistant

Home Assistant saves the logs in a SQLite database by default.  
The recent raw states of sensors are kept in the table ```states``` for about 1-2 month.  
The list of sensors is in ```states_meta```.   
Aggregates are kept in ```statistics_short_term``` (5 minute aggregates stored short-term) and  ```statistics``` (1 hour aggregates stored long-term).  
For the purpose of real time forecast of indoor CO2 levels recent raw values in ```states``` provide the best source of data.  
We use duckdb to extract data from a SQLite database and save the values we need into a dataset for subsequent analysis and processing.  

In [ ]:
# install sqlite

import duckdb

con = duckdb.connect()
con.execute("INSTALL sqlite;")
con.execute("LOAD sqlite;")

In [ ]:
# set up path to data

drive_folder = '/MyDrive/Data/flow-co2/data/'
mount_folder = '/content/drive'
data_folder = mount_folder + drive_folder
print(data_folder)

db_path = data_folder+'home-assistant_v2_config.db'

export_csv = 'values.csv'
export_path = data_folder + export_csv
print(export_path)

In [ ]:
# mount the data source

from google.colab import drive
drive.mount(mount_folder)

In [ ]:
# attach database from config

query = f"ATTACH '{db_path}' AS db_ha (TYPE sqlite);"
con.execute(query)

In [ ]:
# find the range of dates and the number of readings available for each co2 sensor

query = '''
  SELECT
    count(*) as cnt,
    to_timestamp(min(st.last_updated_ts)) as minTS,
    to_timestamp(max(st.last_updated_ts)) as maxTS,
    st.metadata_id,
    sm.entity_id
  FROM db_ha.states st
  INNER JOIN db_ha.states_meta sm ON st.metadata_id = sm.metadata_id
  WHERE st.metadata_id in (171, 561, 159, 597, 576, 146, 153, 142, 101)
  group by st.metadata_id, sm.entity_id
  order by to_timestamp(max(st.last_updated_ts)) desc
  '''
con.sql(query).show(max_rows=600)

In [ ]:
# extract data from the specific sensor sensor.i_9psl_carbon_dioxide which provides more frequent data

query = '''
  SELECT
    st.state as value,
    to_timestamp(st.last_updated_ts) as timestamp
  FROM db_ha.states st
  INNER JOIN db_ha.states_meta sm ON st.metadata_id = sm.metadata_id
  WHERE st.metadata_id in (597)
  order by to_timestamp(st.last_updated_ts) desc
  '''

res = con.sql(query)
res.show(max_rows=10)

In [ ]:
# write sensor readings into a csv file

res.write_csv(export_path)

In [ ]:
# debugging a problem with notebook rendering on github
print('Hello world')